# Violin plot

In [ ]:
# Imports
import numpy as np
import pandas as pd
import os

# figure imports
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Define filenames
# main_dir = '/home/mchevillard/disks/meso_shared'
main_dir = '/home/mszinte/disks/meso_S/data'
project_dir = 'gaze_exp'
pp_dir = "{}/{}/derivatives/pp_data".format(main_dir, project_dir)
tsv_dir ='{}/sub-all/tsv'.format(pp_dir)
fig_dir = '{}/sub-all/figures'.format(pp_dir)
os.makedirs(fig_dir, exist_ok=True)


In [ ]:
# Load data
data = pd.read_table('{}/sub-all_prf_cf.tsv'.format(tsv_dir))
data.drop(columns=['Unnamed: 0'])

In [ ]:
# Filter data
ecc_th = [0, 15]
size_th= [0.1, 20]
rsq_th = [0.05, 1]

# Replace all data outer threshold with NaN data
data.loc[(data.prf_ecc < ecc_th[0]) | (data.prf_ecc > ecc_th[1]) | 
         (data.prf_size < size_th[0]) | (data.prf_size > size_th[1]) | 
         (data.cf_center_ecc < ecc_th[0]) | (data.cf_center_ecc > ecc_th[1]) | 
         (data.cf_center_size < size_th[0]) | (data.cf_center_size > size_th[1]) | 
         (data.prf_rsq_loo <=rsq_th[0]) | (data.cf_center_cf_rsq <=rsq_th[0]) | 
         (data.cf_center_cf_rsq >=rsq_th[1])
        ] = np.nan

data = data.dropna()

rois = pd.unique(data.roi)
mask = pd.notnull(data.subject)
subjects = pd.unique(data.subject[mask])

In [ ]:
# Define colors
roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

In [ ]:
df = data.loc[(data.subject == 'sub-001') & (data.roi == 'V3')]
print(df.cf_center_size.mean())
print(df.prf_size.mean())

## Make Violin plot comparison R2 / CF-projected Size / CF-projected Eccentricity

In [ ]:
rows, cols = 2,2
fig_height, fig_width = 1080,1920

for i, subject in enumerate(subjects):
    fig = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1)
    for j, roi in enumerate(rois):
        
        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        
        # df = df.sort_values('prf_rsq_loo', ascending=False)
        # df = df.head(250)

        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_rsq_loo, name=roi, showlegend=True, legendgroup='avg', points=False, scalemode='width', width=0.75, side='negative', line_color = roi_colors[j], meanline_visible=True), row=1, col=1)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.cf_center_cf_rsq, name=roi, showlegend=True, legendgroup='loo', points=False, scalemode='width', width=0.75, side='positive', line_color = roi_colors[j], meanline_visible=True, fillcolor='rgb(255,255,255)'), row=1, col=1)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_size, name=roi, showlegend=False, legendgroup='avg', points=False, scalemode='width', width=0.75, side='negative', line_color = roi_colors[j], meanline_visible=True), row=1, col=2)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.cf_center_size, name=roi, showlegend=False, legendgroup='loo', points=False, scalemode='width', width=0.75, side='positive', line_color=roi_colors[j], meanline_visible=True, fillcolor='rgb(255,255,255)'), row=1, col=2)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_ecc, name=roi, showlegend=False, legendgroup='avg', points=False, scalemode='width', width=0.75, side='negative', line_color = roi_colors[j], meanline_visible=True), row=2, col=1)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.cf_center_ecc, name=roi, showlegend=False, legendgroup='loo', points=False, scalemode='width', width=0.75, side='positive', line_color = roi_colors[j], meanline_visible=True, fillcolor='rgb(255,255,255)'), row=2, col=1)
        
        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(range=[0,1], title_text='r2', row=1, col=1)
        fig.update_yaxes(range=[0,20], title_text='pRF size (dva)', row=1, col=2)
        fig.update_yaxes(range=[0,15], nticks=4, title_text='pRF eccentricity (dva)', row=2, col=1)
        fig.update_xaxes(showline=True, ticklen=0, linecolor=('rgba(255,255,255,0)'), tickfont=dict(size=18))
        fig.update_traces(spanmode='manual', span=[0,1], row=1, col=1)  
        fig.update_traces(spanmode='manual', span=[0.1,20], row=1, col=2)
        fig.update_traces(spanmode='manual', span=[0,15], row=2, col=1)
    fig.update_layout(height=fig_height, width=fig_width, showlegend=True, legend=dict(orientation="h", yanchor='top', y=1.15, xanchor='left', x=0.22, traceorder='normal', itemwidth=50), template='simple_white', font=dict(size=16))
    fig.show()
    fig.write_image("{}/{}_prf_cf_rsq_size_ecc.pdf".format(fig_dir, subject)) 
    

<!-- ## Violin plot R2 - Visual areas for each subject -->

In [ ]:
# for i, subject in enumerate(subjects):
    
#     for j, roi in enumerate(rois):
#         df = data.loc[(data.subject == subject) & (data.roi == roi)]
#         fig = fig_1_R2 if i==0 else fig_2_R2
        
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_rsq_loo, name=roi, line_color = roi_colors[j], meanline_visible=True))
        
#         # Set axis titles only for the left-most column and bottom-most row
#         fig.update_yaxes(title_text='R2 (leave-one-out)')
#         fig.update_xaxes(title_text='Visual areas')
#         #Add the name of the plot with the subject name
#         fig.add_annotation(x=0.25, y=1, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

#     fig.update_layout(height=fig_height, width=fig_width, template='simple_white')
#     fig.show()
#     fff
#     # fig.write_image('/home/mchevillard/disks/meso_S/{}_r2_roi.pdf'.format(subject))
    

<!-- ## Violin plot Size - Visual areas for each subject -->

In [ ]:
# for i, subject in enumerate(subjects):
#     for j, roi in enumerate(rois):
#         df = data.loc[(data.subject == subject) & (data.roi == roi)]
#         fig=fig_1_SD if i==0 else fig_2_SD
        
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_size, name=roi, line_color = roi_colors[j], meanline_visible=True))

#         # Set axis titles only for the left-most column and bottom-most row
#         fig.update_yaxes(title_text='pRF size (dva)')
#         fig.update_xaxes(title_text='Visual areas')
#         #Add the name of the plot with the subject name
#         fig.add_annotation(x=0.25, y=20, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

#     fig.update_layout(height=fig_height, width=fig_width, template='simple_white')
#     # fig.write_image('/home/mchevillard/disks/meso_S/{}_size_roi.pdf'.format(subject))

<!-- ## Violin plot Eccentricity - Visual areas for each subject -->

In [ ]:
# for i, subject in enumerate(subjects):
#     for j, roi in enumerate(rois):
#         df = data.loc[(data.subject == subject) & (data.roi == roi)]
#         fig=fig_1_ecc if i==0 else fig_2_ecc
        
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_ecc, name=roi, line_color = roi_colors[j], meanline_visible=True))

#         # Set axis titles only for the left-most column and bottom-most row
#         fig.update_yaxes(title_text='pRF eccentricity (dva)')
#         fig.update_xaxes(title_text='Visual areas')
#         #Add the name of the plot with the subject name
#         fig.add_annotation(x=0.25, y=15, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

#     fig.update_layout(height=fig_height, width=fig_width, template='simple_white')
#     # fig.write_image('/home/mchevillard/disks/meso_S/{}_ecc_roi.pdf'.format(subject))

<!-- ## Violin plot shared Eccentricity - Visual areas for each subject -->

In [ ]:
# for i, subject in enumerate(subjects):
#     for j, roi in enumerate(rois):
#         df = data.loc[(data.subject == subject) & (data.roi == roi)]
#         fig=fig_1_ecc_loo if i==0 else fig_2_ecc_loo
        
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_ecc, name=roi, line_color = roi_colors[j], points=False, side='negative', meanline_visible=True))
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.cf_center_ecc, name=roi, line_color = roi_colors[j], points=False, side='positive', meanline_visible=True, fillcolor='rgb(255,255,255)'))

#         # Set axis titles only for the left-most column and bottom-most row
#         fig.update_yaxes(title_text='pRF eccentricity (dva)')
#         fig.update_xaxes(title_text='Visual areas')
#         #Add the name of the plot with the subject name
#         fig.add_annotation(x=0, y=16, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

#     fig.update_layout(violingap=0, violinmode='overlay', height=fig_height, width=fig_width, template='simple_white')
#     fig.show()
#     fff
    # fig.write_image('/home/mchevillard/disks/meso_S/{}_loo_ecc_roi.pdf'.format(subject))

<!-- ## Violin plot pCM - Visual areas for each subject -->

In [ ]:
# for i, subject in enumerate(subjects):
#     for j, roi in enumerate(rois):
#         df = data.loc[(data.subject == subject) & (data.roi == roi)]
#         fig=fig_1_ecc if i==0 else fig_2_ecc
        
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.ecc, name=roi, line_color = roi_colors[j], meanline_visible=True))

#         # Set axis titles only for the left-most column and bottom-most row
#         fig.update_yaxes(title_text='pRF eccentricity (dva)')
#         fig.update_xaxes(title_text='Visual areas')
#         #Add the name of the plot with the subject name
#         fig.add_annotation(x=0.25, y=15, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

#     fig.update_layout(height=fig_height, width=fig_width, template='simple_white')
#     fig.write_image('/home/mchevillard/disks/meso_S/{}_ecc_roi.pdf'.format(subject))

<!-- ## Make subplot -->

In [ ]:
# rows, cols = 2,2
# fig_height, fig_width = 1080,1920

# fig_subplot_1 = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1, shared_xaxes=False)
# fig_subplot_2 = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1, shared_yaxes=False)

# for i, subject in enumerate(subjects):
#     for j, roi in enumerate(rois):
#         fig=fig_subplot_1 if i==0 else fig_subplot_2

#         df = data.loc[(data.subject == subject) & (data.roi == roi)]

#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_rsq_loo, name=roi, showlegend=True, points=False, scalemode='width', width=0.75, line_color = roi_colors[j], meanline_visible=True), row=1, col=1)
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_size, name=roi, showlegend=False, points=False, scalemode='width', width=0.75, line_color = roi_colors[j], meanline_visible=True), row=1, col=2)
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.prf_ecc, name=roi, showlegend=False, points=False, scalemode='width', width=0.75, line_color = roi_colors[j], meanline_visible=True), row=2, col=1)


#         # Set axis titles only for the left-most column and bottom-most row
#         fig.update_yaxes(title_text='r2', row=1, col=1)
#         fig.update_yaxes(title_text='pRF size(dva)', row=1, col=2)
#         fig.update_yaxes(title_text='pRF eccentricity (dva)', row=2, col=1)
#         fig.update_xaxes(title_text='Visual areas', row=2, col=1)

#     fig.update_layout(title= 'pRF parameters VS Visual areas {}'.format(subject), height=fig_height, width=fig_width,showlegend=True, template='simple_white', font=dict(size=16))
#     fig.show()

#     # fig.write_image('/home/mchevillard/disks/meso_S/{}_subplot_pRF.pdf'.format(subject))